In [3]:
import pandas as pd
import numpy as np

### Carga del dataset

In [5]:
df = pd.read_csv("/home/sebas/Integrated-EDA-/data/Gaming_Mental_Health.csv")

### Limpieza de nombres de juegos que será la key para juntarlo con la Api en porcentaje

In [6]:
null_counts = (df.isnull().sum() / len(df)) * 100
print(null_counts)

record_id                            0.0
age                                  0.0
gender                               0.0
daily_gaming_hours                   0.0
game_genre                           0.0
primary_game                         0.0
gaming_platform                      0.0
sleep_hours                          0.0
sleep_quality                        0.0
sleep_disruption_frequency           0.0
academic_work_performance            0.0
grades_gpa                          24.6
work_productivity_score             32.6
mood_state                           0.0
mood_swing_frequency                 0.0
withdrawal_symptoms                  0.0
loss_of_other_interests              0.0
continued_despite_problems           0.0
eye_strain                           0.0
back_neck_pain                       0.0
weight_change_kg                     0.0
exercise_hours_weekly                0.0
social_isolation_score               0.0
face_to_face_social_hours_weekly     0.0
monthly_game_spe

Tratar estas columnas "grades_gpa" "work_productivity_score" lo mas seguro será eliminarlos.

In [7]:
print(df.nunique())

record_id                           1000
age                                   23
gender                                 3
daily_gaming_hours                   132
game_genre                             7
primary_game                          24
gaming_platform                        4
sleep_hours                           61
sleep_quality                          5
sleep_disruption_frequency             5
academic_work_performance              6
grades_gpa                           275
work_productivity_score               10
mood_state                             9
mood_swing_frequency                   5
withdrawal_symptoms                    2
loss_of_other_interests                2
continued_despite_problems             2
eye_strain                             2
back_neck_pain                         2
weight_change_kg                      70
exercise_hours_weekly                 90
social_isolation_score                10
face_to_face_social_hours_weekly     158
monthly_game_spe

Con esto se los valores unicos y tener una ligera idea de las columnas y podemos buscarlos cada columna

No tenemos ninguna fila duplicada.

In [8]:
print(df['gaming_platform'].value_counts())

gaming_platform
Mobile            262
Multi-platform    260
PC                241
Console           237
Name: count, dtype: int64


In [12]:
# En lugar de inventar datos, marcamos los que faltan o imputamos la mediana
df['grades_gpa'] = df['grades_gpa'].fillna(df['grades_gpa'].median())
df['work_productivity_score'] = df['work_productivity_score'].fillna(0) # Asumimos 0 si no hay respuesta

## Dado que muchas de las apis no usan el nombre si no la etiqueta de APPID que descargare en un csv, y le hare un merge a mi dataset, pero primero me quedare solo con los Multi-platform y PC, ya que son los nombres que se encontraran en la api

In [45]:
plataformas = ["PC", "Multi-platform"]

df_filtro_plataforma = df[df["gaming_platform"].isin(plataformas)]
#Ya con esto reduje mi dataset solo a los de PC y Mulitplitaforma


print(df_filtro_plataforma)

    record_id  age  gender  daily_gaming_hours     game_genre  \
0      GD0001   17    Male                11.1   Mobile Games   
1      GD0002   21    Male                 3.0           MOBA   
2      GD0003   23    Male                 7.6            FPS   
3      GD0004   20  Female                 7.2            RPG   
4      GD0005   18    Male                 6.8  Battle Royale   
..        ...  ...     ...                 ...            ...   
992    GD0993   18    Male                 6.1            MMO   
995    GD0996   15  Female                 8.6       Strategy   
996    GD0997   18    Male                 5.4            MMO   
997    GD0998   23    Male                 7.3            RPG   
999    GD1000   29    Male                 3.5            RPG   

             primary_game gaming_platform  sleep_hours sleep_quality  \
0          Clash of Clans              PC          3.7     Very Poor   
1                  Dota 2              PC          7.2          Fair   
2  

### Añadimos la nueva CSV para poder coger los APPID de los juegos y asi facilitar a la hora de buscarlos en la api, asi no tendria que hacer un mapeo, ya que busca por appid ny no por nombre.

In [35]:
df_appid = pd.read_csv("/home/sebas/Integrated-EDA-/data/app_id_info.csv")

In [36]:
df_appid.head()

,appid,Title,Type,Price,Release_Date,Rating,Required_Age,Is_Multiplayer
0,63942,Men of War: Vietnam Special Edition Upgrade Pack,dlc,2.99,1/1/1970 0:00,-1,0,1
1,414120,Modbox,game,14.99,4/5/2016 0:00,-1,0,0
2,363020,IPackThat,game,64.99,10/6/2015 0:00,-1,0,0
3,399550,Gladiators Online - Tiro Pack,dlc,0.99,11/4/2015 0:00,-1,0,0
4,65258,Train Simulator: Settle Carlisle Specials Add-On,dlc,29.99,5/3/2012 0:00,-1,0,0


In [47]:
# Solo quiero el el appid
df_appid_filtrado = df_appid[["appid", "Title"]]
#Para luego haacer un merge, nos aseguramos que ambos esten en el mismo formato

df_filtro_plataforma['primary_key']=df_filtro_plataforma['primary_game'].str.upper().str.strip()

df_appid_filtrado["primary_key"]= df_appid_filtrado["Title"].str.upper().str.strip()



In [ ]:
df_final = pd.merge(
    df_filtro_plataforma, 
    df_appid_filtrado[["appid", "primary_key"]], 
    on='primary_key', 
    how='left'
)

df_final = df_final.drop(columns=['primary_key'])
####Tengo que borrar los NAan

In [51]:
df_final.head()

,record_id,age,gender,daily_gaming_hours,game_genre,primary_game,gaming_platform,sleep_hours,sleep_quality,sleep_disruption_frequency,...,eye_strain,back_neck_pain,weight_change_kg,exercise_hours_weekly,social_isolation_score,face_to_face_social_hours_weekly,monthly_game_spending_usd,years_gaming,gaming_addiction_risk_level,appid
0,GD0001,17,Male,11.1,Mobile Games,Clash of Clans,PC,3.7,Very Poor,Sometimes,...,True,False,6.8,3.7,7,1.3,383.70,3,Severe,NaN
1,GD0002,21,Male,3.0,MOBA,Dota 2,PC,7.2,Fair,Rarely,...,False,False,0.4,8.5,2,10.7,46.64,1,Low,570.0
2,GD0003,23,Male,7.6,FPS,CS:GO,Multi-platform,4.4,Fair,Often,...,False,True,1.8,7.1,5,3.2,100.81,6,Severe,NaN
3,GD0004,20,Female,7.2,RPG,Skyrim,Multi-platform,5.1,Fair,Often,...,True,True,0.2,5.2,4,9.1,51.60,7,High,NaN
4,GD0005,18,Male,6.8,Battle Royale,Apex Legends,PC,3.4,Poor,Never,...,False,False,0.5,6.1,4,4.5,32.57,1,Moderate,NaN


## Conexion con SteamSpy

In [ ]:
import requests
import time
import pandas as pd

def get_steamspy_data(game_name):
    # 1. Intentamos búsqueda por término
    search_url = f"https://steamspy.com/api.php?request=search&term={game_name}"
    
    try:
        response = requests.get(search_url).json()
        
        # SteamSpy devuelve un dict con AppIDs como llaves si encuentra algo
        if response and isinstance(response, dict) and len(response) > 0:
            # Tomamos el primer AppID de los resultados
            first_appid = list(response.keys())[0]
            
            # 2. Pedimos los detalles de ese ID
            detail_url = f"https://steamspy.com/api.php?request=appdetails&appid={first_appid}"
            details = requests.get(detail_url).json()
            
            print(f"✅ Encontrado: {game_name} (AppID: {first_appid})")
            return {
                'primary_game_clean': game_name,
                'average_forever': details.get('average_forever', 0),
                'owners': details.get('owners', '0').replace(',', ''), # Limpiamos comas
                'price': details.get('price', '0')
            }
        else:
            print(f"❌ No encontrado en Steam: {game_name}")
            return None
    except Exception as e:
        print(f"⚠️ Error con {game_name}: {e}")
        return None

# --- EJECUCIÓN ---
# Probamos con una muestra que sepamos que está en Steam para verificar
top_games = df['primary_game'].value_counts().index[:15]
steam_data = []

print("Iniciando consulta a SteamSpy...")
for game in top_games:
    data = get_steamspy_data(game)
    if data:
        steam_data.append(data)
    time.sleep(1.1) # Un poco más de margen por seguridad

df_steam = pd.DataFrame(steam_data)

if df_steam.empty:
    print("\n🚨 ALERTA: df_steam sigue vacío. Es probable que los juegos del top no estén en Steam.")
else:
    print(f"\n✨ Éxito: Se han obtenido datos de {len(df_steam)} juegos.")

Iniciando consulta a SteamSpy...
⚠️ Error con DOTA 2: Expecting value: line 1 column 1 (char 0)
⚠️ Error con ELDEN RING: Expecting value: line 1 column 1 (char 0)
⚠️ Error con STARCRAFT II: Expecting value: line 1 column 1 (char 0)
⚠️ Error con WORLD OF WARCRAFT: Expecting value: line 1 column 1 (char 0)
⚠️ Error con CIVILIZATION VI: Expecting value: line 1 column 1 (char 0)
⚠️ Error con MOBILE LEGENDS: Expecting value: line 1 column 1 (char 0)
⚠️ Error con LEAGUE OF LEGENDS: Expecting value: line 1 column 1 (char 0)
⚠️ Error con FINAL FANTASY XIV: Expecting value: line 1 column 1 (char 0)
⚠️ Error con CYBERPUNK 2077: Expecting value: line 1 column 1 (char 0)
⚠️ Error con ELDER SCROLLS ONLINE: Expecting value: line 1 column 1 (char 0)
⚠️ Error con SKYRIM: Expecting value: line 1 column 1 (char 0)
⚠️ Error con FORTNITE: Expecting value: line 1 column 1 (char 0)
⚠️ Error con CLASH OF CLANS: Expecting value: line 1 column 1 (char 0)
⚠️ Error con WARZONE: Expecting value: line 1 column 1 (

In [ ]:
df_steam.head()

""


In [ ]:
# Unimos por la columna 'primary_game_clean'
df_final = pd.merge(df, df_steam, on='primary_game_clean', how='left')

df_final.to_csv('/home/bigdata/Integrated-EDA-/data/gaming.csv', index=False)

KeyError: 'primary_game_clean'